# Introduccion
Duration: 1:00

Vamos a ver paso a paso la creacion de una app y desplegarla al dispositivo para ejecutarla en el mismo

In [2]:
# %load https://raw.githubusercontent.com/dmpop/geofix/master/geofix.py
import android, sys, os, sqlite3
from datetime import datetime

In [3]:
# Specify wait time and a destination directory
wait = 9000
geofix_dir = '/sdcard/Geofix/'
# Create the destination directories if they don't exist
if not os.path.exists(geofix_dir):
    os.makedirs(geofix_dir)
    os.makedirs(geofix_dir + 'snapshots/')

In [4]:
# Enable location and obtain location data
droid = android.Android()
droid.startLocating()
droid.eventWaitFor('location', int(wait))
location = droid.readLocation().result
droid.stopLocating()

error: [Errno 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión

In [ ]:
# Generate date and time string
dt = datetime.strftime(datetime.now(), '%Y%m%d-%H%M%S')
# Extract latitude and longitude coordinates from the network source
try:
    coords = location['network']
    lat = str(coords['latitude'])
    lon = str(coords['longitude'])
    droid.makeToast('Network coordinates: ' + lat + ' ' + lon)
except (KeyError):
    # If network source is not available, extract latitude and longitude values from GPS
    try:
        coords = location['gps']
        lat = str(coords['latitude'])
        lon = str(coords['longitude'])
        droid.makeToast('GPS coordinates: '+ lat + ' ' + lon)
    except (KeyError):
        droid.makeToast('Geofix failed to obtain coordinates.')
        sys.exit()

In [ ]:
# Generate coordinates in the digiKam-compatible format
digikam = 'geo:' + lat + ',' + lon
# Generate an OpenStreetMap URL and save the prepared data in the geofix.csv file
osm ='http://www.openstreetmap.org/index.html?mlat=' + lat + '&mlon=' + lon + '&zoom=18'
f_path = geofix_dir + 'geofix.csv'
f = open(f_path,'a')
f.write(str(dt) + ', ' + str(lat) + ', ' + str(lon) + ', ' + digikam + ', ' + osm + '\n')
f.close()

In [ ]:
# Save the prepared data in the geofix.sqlite database
if os.path.exists(geofix_dir + 'geofix.sqlite'):
    sql_query = "INSERT INTO geofix (dt, lat, lon, digikam, osm_url) VALUES ('%s', '%s', '%s', '%s', '%s')" % (dt, lat, lon, digikam, osm)
    conn = sqlite3.connect(geofix_dir + 'geofix.sqlite')
    conn.execute(sql_query)
    conn.commit()
    conn.close()
else:
    # Create the database if it doesn't exist
    conn = sqlite3.connect(geofix_dir + 'geofix.sqlite')
    conn.execute("CREATE TABLE geofix (id INTEGER PRIMARY KEY, dt VARCHAR, lat VARCHAR, lon VARCHAR, digikam VARCHAR, osm_url VARCHAR)")
    sql_query = "INSERT INTO geofix (dt, lat, lon, digikam, osm_url) VALUES ('%s', '%s', '%s', '%s', '%s')" % (dt, lat, lon, digikam, osm)
    conn.execute(sql_query)
    conn.commit()
    conn.close()

In [ ]:
# Take a photo
droid.cameraInteractiveCapturePicture(geofix_dir + 'snapshots/' + dt + '.jpg')
droid.makeToast('All done!')
droid.vibrate()